In [4]:
import pandas as pd

# 1. 데이터 로드
matching_df = pd.read_csv('matching_questions_cnn_953.csv')
prices_df = pd.read_csv('question_prices_with_changes_cnn.csv')

# 2. 필요한 열 선택
prices_subset = prices_df[['yes_price', 'change_1d', 'change_3d', 'change_5d']]

# 3. 열 옆으로 붙이기
# axis=1을 사용해 수평으로 결합
combined_df = pd.concat([matching_df, prices_subset], axis=1)

# 4. 결과를 새로운 CSV로 저장
combined_df.to_csv('cnn_with_labels.csv', index=False)
print("저장되었습니다.")

저장되었습니다.
